# Week 8 Module 4 Assignment 2
## Francis Yang 11/25/2022
### Face Recognition (from Python Data Science Handbook by Jake VanderPlas)

As an example of support vector machines in action, let's take a look at the facial recognition problem. We will use the Labeled Faces in the Wild dataset, which consists of several thousand collated photos of various public figures. A fetcher for the dataset is built into Scikit-Learn:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import fetch_lfw_people

faces = fetch_lfw_people(min_faces_per_person=60)

print(faces.target_names)
print(faces.images.shape)

['Ariel Sharon' 'Colin Powell' 'Donald Rumsfeld' 'George W Bush'
 'Gerhard Schroeder' 'Hugo Chavez' 'Junichiro Koizumi' 'Tony Blair']
(1348, 62, 47)


Each image contains [62×47] or nearly 3,000 pixels. We could proceed by simply using each pixel value as a feature, but often it is more effective to use some sort of preprocessor to extract more meaningful features; here we will use a principal component analysis (we will learn about PCA later) to extract 150 fundamental components to feed into our support vector machine classifier. We can do this most straightforwardly by packaging the preprocessor and the classifier into a single pipeline:

In [16]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pca = PCA(n_components=150, whiten=True, random_state=42)
svc = SVC()

pipe = Pipeline([('pca',pca),('estimator',svc)])

Finally, we can use a grid search cross-validation to explore combinations of parameters. Here we will adjust C (which controls the margin hardness) and gamma (which controls the size of the radial basis function kernel), and determine the best model:

In [17]:
faces.keys()

dict_keys(['data', 'images', 'target', 'target_names', 'DESCR'])

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(
    faces.data, faces.target, random_state=1)

param_grid = {'estimator__C': np.arange(1, 10, 2), 'estimator__gamma': np.linspace(0.1, 0.9, num=9)}
grid = GridSearchCV(pipe, param_grid, cv=5)
grid.fit(X_train, y_train)

grid.best_params_

{'estimator__C': 1, 'estimator__gamma': 0.1}

In [21]:
vars(grid)

{'scoring': None,
 'estimator': Pipeline(steps=[('pca', PCA(n_components=150, random_state=42, whiten=True)),
                 ('estimator', SVC())]),
 'n_jobs': None,
 'refit': True,
 'cv': 5,
 'verbose': 0,
 'pre_dispatch': '2*n_jobs',
 'error_score': nan,
 'return_train_score': False,
 'param_grid': {'estimator__C': array([1, 3, 5, 7, 9]),
  'estimator__gamma': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])},
 'multimetric_': False,
 'best_index_': 0,
 'best_score_': 0.3916938984538848,
 'best_params_': {'estimator__C': 1, 'estimator__gamma': 0.1},
 'best_estimator_': Pipeline(steps=[('pca', PCA(n_components=150, random_state=42, whiten=True)),
                 ('estimator', SVC(C=1, gamma=0.1))]),
 'refit_time_': 0.27382993698120117,
 'scorer_': <function sklearn.metrics._scorer._passthrough_scorer(estimator, *args, **kwargs)>,
 'cv_results_': {'mean_fit_time': array([0.21890512, 0.2080575 , 0.21933241, 0.22287793, 0.21892395,
         0.1952878 , 0.21751051, 0.22355318, 0.21

In [ ]:
#score = 0.39169